# Qlib Stock Analysis Implementation

This notebook demonstrates how to use Microsoft's Qlib library for quantitative investment analysis.

First, we'll install the required packages and set up our environment.

In [ ]:
!pip install numpy pandas scikit-learn torch
!pip install --upgrade pip setuptools wheel
!pip install pyqlib

In [ ]:
import qlib
from qlib.config import REG_CN
from qlib.data import D
from qlib.utils import init_instance_by_config
import pandas as pd
import numpy as np

## Download and Initialize Data

We'll download the China stock market data and initialize Qlib.

In [ ]:
# Download China stock market data
!python -m qlib.run.get_data qlib_data --target_dir ~/.qlib/qlib_data/cn_data --region cn

# Initialize Qlib with China stock market data
qlib.init(provider_uri='~/.qlib/qlib_data/cn_data', region=REG_CN)

## Basic Data Analysis

Let's analyze some basic stock data using Qlib's features.

In [ ]:
def analyze_stock_data(stock_id=["SH.600000"], start_time='2023-01-01', end_time='2023-12-31'):
    """Analyze basic stock data"""
    # Define the features we want to analyze
    fields = ["$close", "$open", "$high", "$low", "$volume"]
    
    # Get the features data
    data = D.features(
        instruments=stock_id,  # Pass as a list
        fields=fields,
        freq='day',
        start_time=start_time,
        end_time=end_time,
        inst_processors=None  # Explicitly set to None to avoid the multiple values error
    )
    
    # Convert to DataFrame for analysis - modified to handle list of instruments
    df = data.loc[stock_id[0]].reset_index()  # Access first stock in the list
    
    # Calculate basic statistics
    print("\nBasic Statistics:")
    print(f"Number of trading days: {len(df)}")
    
    print("\nPrice Summary:")
    print(df[["$close", "$open", "$high", "$low"]].describe())
    
    print("\nVolume Summary:")
    print(df["$volume"].describe())
    
    # Calculate daily returns
    df['daily_return'] = df['$close'].pct_change()
    
    print("\nDaily Returns Summary:")
    print(df['daily_return'].describe())
    
    return df

# Analyze a specific stock - pass the stock ID as a list with correct format
df = analyze_stock_data(stock_id=["SH.600000"])

## Data Visualization

Let's create some visualizations of our stock data.

In [ ]:
import matplotlib.pyplot as plt

def plot_stock_analysis(df):
    # Create a figure with multiple subplots
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(15, 12))
    
    # Plot 1: Price Movement
    ax1.plot(df.index, df['$close'], label='Close')
    ax1.plot(df.index, df['$open'], label='Open')
    ax1.set_title('Stock Price Movement')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Price')
    ax1.legend()
    
    # Plot 2: Volume
    ax2.bar(df.index, df['$volume'])
    ax2.set_title('Trading Volume')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Volume')
    
    # Plot 3: Daily Returns
    ax3.hist(df['daily_return'].dropna(), bins=50)
    ax3.set_title('Distribution of Daily Returns')
    ax3.set_xlabel('Daily Return')
    ax3.set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

# Create visualizations
plot_stock_analysis(df)

## Advanced Analysis: Moving Averages and Technical Indicators

In [ ]:
def calculate_technical_indicators(df):
    # Calculate moving averages
    df['MA5'] = df['$close'].rolling(window=5).mean()
    df['MA20'] = df['$close'].rolling(window=20).mean()
    
    # Calculate RSI
    delta = df['$close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Plot technical indicators
    plt.figure(figsize=(15, 10))
    
    # Price and Moving Averages
    plt.subplot(2, 1, 1)
    plt.plot(df.index, df['$close'], label='Close Price')
    plt.plot(df.index, df['MA5'], label='5-day MA')
    plt.plot(df.index, df['MA20'], label='20-day MA')
    plt.title('Price and Moving Averages')
    plt.legend()
    
    # RSI
    plt.subplot(2, 1, 2)
    plt.plot(df.index, df['RSI'])
    plt.axhline(y=70, color='r', linestyle='--')
    plt.axhline(y=30, color='g', linestyle='--')
    plt.title('Relative Strength Index (RSI)')
    
    plt.tight_layout()
    plt.show()
    
    return df

# Calculate and plot technical indicators
df_with_indicators = calculate_technical_indicators(df)